In [2]:
import pyaudio
import struct
import numpy as np
import matplotlib.pyplot as plt
import time
%matplotlib tk
from pycaw.pycaw import AudioUtilities,IAudioEndpointVolume
from ctypes import cast,POINTER
import comtypes
import scipy.io.wavfile as wav
import scipy.signal as sig
import math

In [3]:
%matplotlib tk

In [4]:
audioSample = 1024*4
Format = pyaudio.paInt16
channels = 2
rate = 44100

In [7]:
def initialiseStream():
    p = pyaudio.PyAudio()
    stream = p.open(
                format=Format,
                channels=2,
                rate=rate,
                input=True,
                frames_per_buffer=audioSample,
                input_device_index=2)
    return stream

def initialiseFigures(portraitOrLandscape = 'p'):
    if portraitOrLandscape == 'p' or portraitOrLandscape == 'P':
        fig,(ax1,ax2,ax3) = plt.subplots(3, 1)
    else:
        fig,(ax1,ax2,ax3) = plt.subplots(1,3)

    ax1.set_xlim(0, audioSample)
    ax1.set_ylim(-0.15,0.15)
    return (fig,ax1,ax2,ax3)
    
fig,ax1,ax2,ax3 = initialiseFigures('l')
x = np.arange(0,2*audioSample,2)

line, = ax1.plot(x,np.random.rand(audioSample))
line2, = ax1.plot(x,np.random.rand(audioSample), color='orange')

stream = initialiseStream()
data = stream.read(audioSample)
dayta = np.reshape(np.frombuffer(data, dtype=np.int16), (audioSample, 2))
NPERSEG = 512
f,t,Sxx = sig.spectrogram(dayta[:,0], fs=rate, window="hann",
                              nperseg=NPERSEG)
ax3.set_ylim([0,5000])
y = ax3.pcolormesh(t, f, 10*np.log10(Sxx), vmin=-20, vmax=60)


ex = np.fft.rfft(dayta[:,0])
freqs = np.fft.fftfreq(ex.size)
mag = np.abs(ex)


line3, = ax2.plot(np.fft.fftshift(freqs),10*np.log10(mag))
ax2.set_ylim([0,150])
ax1.axis('off')
ax2.axis('off')
ax3.axis('off')
ax1.set_facecolor('k')
ax2.set_facecolor('k')
ax3.set_facecolor('k')

plt.ion()
plt.show()

INT16_MAX = 32767

while True:
    time.sleep(0.04)
    data = stream.read(audioSample)
    dayta = np.reshape(np.frombuffer(data, dtype=np.int16), (audioSample, 2))
    
    ax1.set_ylim(-np.max(dayta)/INT16_MAX,np.max(dayta)/INT16_MAX)
    line.set_ydata(dayta[:,0]/INT16_MAX)
    line2.set_ydata(dayta[:,1]/INT16_MAX)
    
    ex = np.fft.rfft(dayta[:,0])
    freqs = np.fft.fftfreq(ex.size)
    ax2.set_ylim(0,np.max(10*np.log10(np.abs(ex))))
    line3.set_data([np.fft.fftshift(freqs)],[10*np.log10(np.maximum(np.abs(ex), 1e-10))])
    
    f,t,Sxx = sig.spectrogram(dayta[:,0], fs=rate, window="hann",
                              nperseg=NPERSEG)
    y.set_array(10*np.log10(Sxx).ravel())
    
    fig.canvas.draw()
    fig.canvas.flush_events()



C:\Users\Michael\Anaconda3\lib\site-packages\ipykernel_launcher.py:67: RuntimeWarning: divide by zero encountered in log10


TclError: invalid command name "pyimage21"